# Composite "Closest Date"
Takes the last image of the given collection and fills the masked pixels with the last available pixel

In [ ]:
import ee
ee.Initialize()

In [ ]:
from geetools import tools, composite, cloud_mask, algorithms

In [ ]:
import geedatasets
import ipygee as ui

In [ ]:
Map = ui.Map()
Map.show()

In [ ]:
l8 = geedatasets.landsat.Landsat8SR()

In [ ]:
vis = l8.visualization('NSR')

In [ ]:
l8col = l8.collection()

In [ ]:
p = ee.Geometry.Point([-70.72, -41.92]).buffer(50000)

In [ ]:
Map.addLayer(p)

In [ ]:
Map.centerObject(p, zoom=8)

### add date band

In [ ]:
col = l8col.map(lambda img: img.addBands(tools.date.makeDateBand(img)))

### Filter and mask clouds

In [ ]:
col = col.filterBounds(p)\
         .filterDate('2017-01-01', '2017-03-01')\
         .map(lambda img: l8.applyMask(img, 'pixel_qa', ['cloud', 'shadow', 'snow']))

In [ ]:
ui.eprint(col.size())

### add all images to the map

In [ ]:
Map.addImageCollection(col, vis, '{system_date} {id}')

## Make composite

In [ ]:
comp = composite.closestDate(col)

In [ ]:
Map.addLayer(comp, vis, 'composite')

In [ ]:
Map.addLayer(comp.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates')

### clip to first image (target date)

In [ ]:
comp_clip = composite.closestDate(col, clip_to_first=True)

In [ ]:
Map.addLayer(comp_clip, vis, 'composite 2017-02-25 clip to first')